<a href="https://colab.research.google.com/github/nebojsa-bozanic/med-flamingo/blob/master/med_flamingo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/snap-stanford/med-flamingo.git

Cloning into 'med-flamingo'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 30 (delta 6), reused 26 (delta 3), pack-reused 0
Receiving objects: 100% (30/30), 426.85 KiB | 22.47 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [2]:
!git lfs install
%cd /content/med-flamingo
!mkdir models
%cd /content/med-flamingo/models
!git clone https://huggingface.co/decapoda-research/llama-7b-hf

Git LFS initialized.
/content/med-flamingo
/content/med-flamingo/models
Cloning into 'llama-7b-hf'...
remote: Enumerating objects: 91, done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 91
Unpacking objects: 100% (91/91), 22.58 KiB | 1.33 MiB/s, done.
Filtering content: 100% (34/34), 12.55 GiB | 155.13 MiB/s, done.


In [3]:
!pip install huggingface-hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 117.2 kB/s eta 0:00:00


In [4]:
!pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-xtpevwoj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-xtpevwoj
  Resolved https://github.com/huggingface/accelerate to commit 21d127334ebbcdd4f2f0695a5b36753dae6d747b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.22.0.dev0-py3-none-any.whl size=250948 sha256=611d991e53dd4f4c8610e6cb980e72f809cf968667b835d16ff220838b0d21bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-vpf8pfwb/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


In [5]:
!pip install git+https://github.com/huggingface/transformers@01734dba842c29408c96caa5c345c9e415c7569b

  Cloning https://github.com/huggingface/transformers (to revision 01734dba842c29408c96caa5c345c9e415c7569b) to /tmp/pip-req-build-aqo38adw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-aqo38adw
  Running command git rev-parse -q --verify 'sha^01734dba842c29408c96caa5c345c9e415c7569b'
  Running command git fetch -q https://github.com/huggingface/transformers 01734dba842c29408c96caa5c345c9e415c7569b
  Running command git checkout -q 01734dba842c29408c96caa5c345c9e415c7569b
  Resolved https://github.com/huggingface/transformers to commit 01734dba842c29408c96caa5c345c9e415c7569b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.29.0.dev0-py3-none-any.whl size=7011470 sha256=e9cafbdc43864d0d31313bea5c

In [6]:
!pip install sentencepiece==0.1.98

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.4 MB/s eta 0:00:00


In [7]:
!pip install git+https://github.com/usuyama/open_clip.git@01a53cc46662d8b28da0c9d73271bfb8f6f4b20d

  Cloning https://github.com/usuyama/open_clip.git (to revision 01a53cc46662d8b28da0c9d73271bfb8f6f4b20d) to /tmp/pip-req-build-orr3tpoo
  Running command git clone --filter=blob:none --quiet https://github.com/usuyama/open_clip.git /tmp/pip-req-build-orr3tpoo
  Running command git rev-parse -q --verify 'sha^01a53cc46662d8b28da0c9d73271bfb8f6f4b20d'
  Running command git fetch -q https://github.com/usuyama/open_clip.git 01a53cc46662d8b28da0c9d73271bfb8f6f4b20d
  Running command git checkout -q 01a53cc46662d8b28da0c9d73271bfb8f6f4b20d
  Resolved https://github.com/usuyama/open_clip.git to commit 01a53cc46662d8b28da0c9d73271bfb8f6f4b20d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.4 MB/s eta 0:00:00
  Created wheel for open-clip-torch: filename=open_clip_torch-2.16.0-py3-none-

In [8]:
!pip install open-flamingo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.7 MB/s eta 0:00:00


```
change "tokenizer_class": from "LLaMAToenizer" to "LlamaTokenizer" in /content/med-flamingo/models/llama-7b-hf/tokenizer_config.json
```

In [10]:
%cd /content/med-flamingo/scripts
!python demo.py

/content/med-flamingo/scripts
2023-08-17 21:56:03.696643: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading model..
/usr/local/lib/python3.10/dist-packages/open_clip/pretrained.py:309: UserWarning: /root/.cache/clip/ViT-L-14.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(f"{download_target} exists, but the SHA256 checksum does not match; re-downloading the file")
100%|████████████████████████████████████████| 933M/933M [00:07<00:00, 130MiB/s]
Using pad_token, but it is not set yet.
Loading checkpoint shards: 100% 33/33 [00:14<00:00,  2.24it/s]
Flamingo model initialized with 1309919248 trainable parameters
Downloaded Med-Flamingo checkpoint to /root/.cache/huggingface/hub/models--med-flamingo--med-flamingo/snapshots/7243cd83bd426ceade9c4de9844cc5e5f3ff75e0/model.pt
Preprocess data
Generate from multimodal few-shot prompt
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils